In [1]:
import zipfile
import csv 
import pandas as pd
import sklearn
from sklearn.preprocessing import MinMaxScaler
import os
import numpy as np
import urllib
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import (RandomForestRegressor , RandomForestClassifier , ExtraTreesRegressor , ExtraTreesClassifier)
import numpy as np
import pickle
from sklearn.metrics import (mean_squared_error,mean_absolute_error,r2_score , accuracy_score ,classification_report,silhouette_score) 
from sklearn.cross_validation import train_test_split
from sklearn import neural_network
from sklearn.neural_network import MLPRegressor ,MLPClassifier
from sklearn import svm 
from sklearn.svm import SVC ,SVR
from sklearn.neighbors import KNeighborsClassifier , KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
import boto3
import sys
import datetime
from sklearn.pipeline import Pipeline ,make_pipeline
from sklearn.preprocessing import LabelEncoder
import plotly.plotly as py
import plotly.graph_objs as go
from sklearn.multioutput import MultiOutputRegressor

C:\Users\Dell-pc\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [16]:
if not os.path.exists(r'C:\ASSIGNMENT_3_PROPULSIONS'):
    os.makedirs(r'C:\ASSIGNMENT_3_PROPULSIONS')

In [17]:
#import urllib
#from urllib import request
#response = urllib.request.urlopen('http://www.website.com/log.zip')
#zipcontent= response.read()
#[WinError 10054] An existing connection was forcibly closed by the remote host

zip = zipfile.ZipFile(r'C:\ASSIGNMENT_3_PROPULSIONS\UCI+CBM+Dataset.zip')  
zip.extractall(r'C:\ASSIGNMENT_3_PROPULSIONS')

In [18]:
### READING THE DATASET AD DATAFRAME ###
f = open(r'C:\ASSIGNMENT_3_PROPULSIONS\UCI CBM Dataset\data.txt' , 'r')
my_data = f.readlines() #converting into list
my_arr = np.array(my_data) # making it into numpy array

arr = []
for k in range(11934):
    i = 3
    while(i<289):
        temp = float(my_arr[k][i:i+13])
        arr.append(temp)
        i = i+16

arr = np.array(arr)
arr = arr.reshape(11934,18)

my_final_data = pd.DataFrame(arr , columns = ['1 - Lever position (lp) [ ]', '2 - Ship speed (v) [knots]', 
                                              '3 - Gas Turbine shaft torque (GTT) [kN m]',
                                              '4 - Gas Turbine rate of revolutions (GTn) [rpm]', 
                                              '5 - Gas Generator rate of revolutions (GGn) [rpm]', 
                                              '6 - Starboard Propeller Torque (Ts) [kN]', '7 - Port Propeller Torque (Tp) [kN]', 
                                              '8 - HP Turbine exit temperature (T48) [C]', 
                                              '9 - GT Compressor inlet air temperature (T1) [C]',
                                              '10 - GT Compressor outlet air temperature (T2) [C]', 
                                              '11 - HP Turbine exit pressure (P48) [bar]', 
                                              '12 - GT Compressor inlet air pressure (P1) [bar]',
                                              '13 - GT Compressor outlet air pressure (P2) [bar]', 
                                              '14 - Gas Turbine exhaust gas pressure (Pexh) [bar]',
                                              '15 - Turbine Injecton Control (TIC) [%]', '16 - Fuel flow (mf) [kg/s]', 
                                              '17 - GT Compressor decay state coefficient.', 
                                              '18 - GT Turbine decay state coefficient.'])

my_final_data.to_csv(r'C:\ASSIGNMENT_3_PROPULSIONS\UCI CBM Dataset\final_dataset.csv')


In [19]:
### CLEANING THE DATASET ###

df = pd.read_csv(r'C:\ASSIGNMENT_3_PROPULSIONS\UCI CBM Dataset\final_dataset.csv')
num_of_missing_values = df.isnull().sum().sum()
if ( num_of_missing_values == 0) :
    final_file = df
else :
    final_file = df.fillna(method='bfill' ,axis = 0).fillna('0')

final_file.to_csv(r'C:\ASSIGNMENT_3_PROPULSIONS\UCI CBM Dataset\final_file.csv',index=False)

In [21]:
### FEATURE ENGINEERING ###

df_fe = pd.read_csv(r'C:\ASSIGNMENT_3_PROPULSIONS\UCI CBM Dataset\final_file.csv')
df_fe = df_fe.drop('Unnamed: 0',axis=1)
#df= df.drop('Unnamed: 0.1',axis = 1)
#print(df_1)
scaler = MinMaxScaler()
scale_data = scaler.fit_transform(df_fe)
scaled_data = pd.DataFrame(scale_data , columns = ['1 - Lever position (lp) [ ]', '2 - Ship speed (v) [knots]', 
                                          '3 - Gas Turbine shaft torque (GTT) [kN m]',
                                          '4 - Gas Turbine rate of revolutions (GTn) [rpm]', 
                                          '5 - Gas Generator rate of revolutions (GGn) [rpm]', 
                                          '6 - Starboard Propeller Torque (Ts) [kN]', '7 - Port Propeller Torque (Tp) [kN]', 
                                          '8 - HP Turbine exit temperature (T48) [C]', 
                                          '9 - GT Compressor inlet air temperature (T1) [C]',
                                          '10 - GT Compressor outlet air temperature (T2) [C]', 
                                          '11 - HP Turbine exit pressure (P48) [bar]', 
                                          '12 - GT Compressor inlet air pressure (P1) [bar]',
                                          '13 - GT Compressor outlet air pressure (P2) [bar]', 
                                          '14 - Gas Turbine exhaust gas pressure (Pexh) [bar]',
                                          '15 - Turbine Injecton Control (TIC) [%]', '16 - Fuel flow (mf) [kg/s]', 
                                          '17 - GT Compressor decay state coefficient.', 
                                          '18 - GT Turbine decay state coefficient.'])

scaled_data.to_csv(r'C:\ASSIGNMENT_3_PROPULSIONS\UCI CBM Dataset\final_fe.csv',index=False)


In [34]:
### PICKLING MODELS ###

df_lr = pd.read_csv(r'C:\ASSIGNMENT_3_PROPULSIONS\UCI CBM Dataset\final_fe.csv')
df_split = np.split(df_lr ,[16],axis = 1) # splitting the dataframe to produce 2 dataframes. One with variables and
#df_split = np.split(df_split[1],[16],axis = 1 )
df_variables = df_split[0]
#df_variables1 = np.array(df_variables)
#df_variables2 = df_variables1.reshape(1,10608)
#df_variables = pd.DataFrame(df_variables)
#print(df_variables)
# other with taret variables.
df_target = df_split[1]
df_target1 = np.split(df_target,[1],axis = 1)
df_target2 = df_target1[0].values
df_target3 = df_target1[1].values
X = df_variables
Y = df_target


train_X,test_X ,train_Y,test_Y = train_test_split(X, Y, test_size=0.30,random_state=1)

### LINEAR REGRESSION ###
mlr = LinearRegression()
#mode1 = MultiOutputRegressor(mlr(fit(train_X , train_Y)))
mlr.fit(train_X , train_Y)
pickle.dump(mlr ,open(r'C:\ASSIGNMENT_3_PROPULSIONS\UCI CBM Dataset\Linear_Regression.pkl', "wb" ))

### RANDOM FOREST REGRESSOR ###
rfr = RandomForestRegressor(n_estimators =100, random_state = 1,n_jobs=-1)
rfr.fit(train_X, train_Y)
pickle.dump(rfr ,open(r'C:\ASSIGNMENT_3_PROPULSIONS\UCI CBM Dataset\Random_Forest_Regression.pkl', "wb" ))

### K-NEAREST NEIGHBORS ###
knn_reg = KNeighborsRegressor(n_neighbors=11, p=2, weights="distance")
knn_reg.fit(train_X, train_Y)
pickle.dump(knn_reg ,open(r'C:\ASSIGNMENT_3_PROPULSIONS\UCI CBM Dataset\KNN.pkl', "wb" ))

In [ ]:
### S3 UPLOAD ###
buck_name=" " #enter bucket name

S3_client = boto3.client('s3', Input_location, aws_access_key_id= Access_key, aws_secret_access_key= Secret_key)

if Input_location == 'us-east-1':
        S3_client.create_bucket(
            Bucket=buck_name
        )
else:
    S3_client.create_bucket(
            Bucket=buck_name,
            CreateBucketConfiguration={'LocationConstraint': Input_location},
        )

S3_client.upload_file(r'C:\ASSIGNMENT_3_PROPULSIONS\UCI CBM Dataset\Linear_Regression.pkl', buck_name, r'C:\ASSIGNMENT_3_PROPULSIONS\UCI CBM Dataset\Linear_Regression.pkl')
S3_client.upload_file(r'C:\ASSIGNMENT_3_PROPULSIONS\UCI CBM Dataset\Random_Forest_Regression.pkl', buck_name, r'C:\ASSIGNMENT_3_PROPULSIONS\UCI CBM Dataset\Random_Forest_Regression.pkl')
S3_client.upload_file(r'C:\ASSIGNMENT_3_PROPULSIONS\UCI CBM Dataset\KNN.pkl', buck_name, r'C:\ASSIGNMENT_3_PROPULSIONS\UCI CBM Dataset\KNN.pkl')